In [1]:
import random
import json
import torch
from model import NeuralNet
from nltk_utils import bag_of_words, tokenize

How are you'll?
['How', 'are', 'you', "'ll", '?']
['organ', 'organ', 'organ']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ishas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open('intents.json','r') as f:
    intents_dict = json.load(f)
FILE ="data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()



NeuralNet(
  (l1): Linear(in_features=56, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [4]:
bot_name = "Ishika"
print("Let's chat! type 'quit' to exit")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    output = model(X)
    _, predicted = torch.max(output,dim = 1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim = 1)
    prob = probs[0][predicted.item()]
    
    if(prob.item()>0.75):
        for intent in intents_dict["intents"]:
            if(tag == intent["tag"]):
                print(f"{bot_name}:{random.choice(intent['responses'])}")
    else:
        print(f'{bot_name}:I do not understand....')
        

    

Let's chat! type 'quit' to exit
You: hi
Ishika:Hi there! What can I do for you?
You: can i buy a tshirt from your company?
Ishika:We accept VISA, Mastercard and Paypal.However, we do not accept cash.
You: what can i buy?
Ishika:I do not understand....
You: what do you sell?
Ishika:We sell tshirts and tops for ladies.
You: how can i pay?
Ishika:We accept VISA, Mastercard and Paypal.However, we do not accept cash.
You: tell me a joke
Ishika:What did the buffalo say when his son left for college? Bison.
You: tell me another joke
Ishika:Why did the hipster burn his mouth? He drank the coffee before it was cool.
You: how many jokes do you know
Ishika:We accept most major credit cards, and Paypal.However, we do not accept cash.
You: quit
